In [63]:
import matplotlib.pyplot as plt
import numpy as np
import pyaudio
from pyqtgraph.Qt import QtGui, QtCore
import pyqtgraph as pg
import struct
from scipy.fftpack import fft
import sys
import time
import serial

%matplotlib tk

In [164]:
class AudioStream(object):
    def __init__(self):

        # stream constants
        self.CHUNK = 1024 * 4
        self.FORMAT = pyaudio.paInt16
        self.CHANNELS = 1
        self.RATE = 44100
        self.pause = False

        # stream object
        self.p = pyaudio.PyAudio()
        self.stream = self.p.open(
            format=self.FORMAT,
            channels=self.CHANNELS,
            rate=self.RATE,
            input=True,
            output=True,
            frames_per_buffer=self.CHUNK,
        )
        self.init_plots()
        self.start_plot()

    def init_plots(self):

        # x variables for plotting
        x = np.arange(0, 2 * self.CHUNK, 2)
        xf = np.linspace(0, self.RATE, self.CHUNK)

        # create matplotlib figure and axes
        self.fig, (ax1, ax2) = plt.subplots(2, figsize=(15, 7))
        self.fig.canvas.mpl_connect('button_press_event', self.onClick)

        # create a line object with random data
        self.line, = ax1.plot(x, np.random.rand(self.CHUNK), '-', lw=2)

        # create semilogx line for spectrum
        self.line_fft, = ax2.plot(
            xf, np.random.rand(self.CHUNK), '-', lw=2)

        # format waveform axes
        ax1.set_title('Audio Waveform and Spectrum')
        ax1.set_xlabel('samples')
        ax1.set_ylabel('volume')
        ax1.set_ylim(0, 255)
        ax1.set_xlim(0, 2 * self.CHUNK)
        plt.setp(
            ax1, yticks=[0, 128, 255],
            xticks=[0, self.CHUNK, 2 * self.CHUNK],
        )
        plt.setp(ax2, yticks=[0, 1],)

        # format spectrum axes
        ax2.set_xlabel('freq (Hz)')
        ax2.set_ylabel('Energy (normalized)')
        ax2.set_xlim(20, 5e3)

        # show axes
        thismanager = plt.get_current_fig_manager()
#         thismanager.window.setGeometry(5, 120, 1910, 1070)
        plt.show(block=False)

    def start_plot(self):

        print('stream started')
        frame_count = 0
        start_time = time.time()
        try:
            while not self.pause:
                data = self.stream.read(self.CHUNK)
                print(type(data))
                data_int = struct.unpack(str(2 * self.CHUNK) + 'B', data)
                data_np = np.array(data_int, dtype='b')[::2] + 128

                self.line.set_ydata(data_np)

                # compute FFT and update line
                yf = fft(data_int)
                self.line_fft.set_ydata(
                    np.abs(yf[0:self.CHUNK]) / (128 * self.CHUNK))

                # update figure canvas
                self.fig.canvas.draw()
                self.fig.canvas.flush_events()
                frame_count += 1
        except:
            self.fr = frame_count / (time.time() - start_time)
            print('average frame rate = {:.0f} FPS'.format(self.fr))
            self.exit_app()

    def exit_app(self):
        print('stream closed')
        self.p.close(self.stream)

    def onClick(self, event):
        self.pause = True

In [165]:
AudioStream()

stream started
<class 'bytes'>
<class 'bytes'>
<class 'bytes'>
<class 'bytes'>
<class 'bytes'>
<class 'bytes'>
<class 'bytes'>
<class 'bytes'>
<class 'bytes'>
<class 'bytes'>
<class 'bytes'>
<class 'bytes'>
<class 'bytes'>
<class 'bytes'>
<class 'bytes'>
<class 'bytes'>
<class 'bytes'>
<class 'bytes'>
<class 'bytes'>
<class 'bytes'>
<class 'bytes'>
<class 'bytes'>
average frame rate = 8 FPS
stream closed


In [126]:
ser = serial.Serial('COM12', 115200)

class SerialStream(object):
    def __init__(self):

        # stream constants
        self.CHUNK = 256
        self.FORMAT = pyaudio.paInt16
        self.CHANNELS = 1
        self.RATE = 1000
        self.pause = False

        # stream object
        self.p = pyaudio.PyAudio()
        self.stream = self.p.open(
            format=self.FORMAT,
            channels=self.CHANNELS,
            rate=self.RATE,
            input=True,
            output=True,
            frames_per_buffer=self.CHUNK,
        )
        self.init_plots()
        self.start_plot()

    def init_plots(self):

        # x variables for plotting
        x = np.arange(0, 2 * self.CHUNK, 2)
        xf = np.linspace(0, self.RATE, self.CHUNK)

        # create matplotlib figure and axes
        self.fig, (ax1, ax2) = plt.subplots(2, figsize=(15, 9))
        self.fig.canvas.mpl_connect('button_press_event', self.onClick)

        # create a line object with random data
        self.line, = ax1.plot(x, np.random.rand(self.CHUNK), '-', lw=2)

        # create semilogx line for spectrum
        self.line_fft, = ax2.semilogx(
            xf, np.random.rand(self.CHUNK), '-', lw=2)
        

        # format waveform axes
        ax1.set_title('AUDIO WAVEFORM')
        ax1.set_xlabel('samples')
        ax1.set_ylabel('volume')
        ax1.set_ylim(0, 3000)
        ax1.set_xlim(0, 2 * self.CHUNK)
        plt.setp(
            ax1, yticks=[0, 128, 255],
            xticks=[0, self.CHUNK, 2 * self.CHUNK],
        )
        plt.setp(ax2, yticks=[0, 1],)

        # format spectrum axes
        ax2.set_xlim(20, self.RATE / 2)

        # show axes
        thismanager = plt.get_current_fig_manager()
#         thismanager.window.setGeometry(5, 120, 1910, 1070)
        plt.show(block=False)

    def start_plot(self):

        print('stream started')
        frame_count = 0
        start_time = time.time()
        try:
            while not self.pause:
                data = self.sample(self.CHUNK)
                data_int = data
                data_np = data

                self.line.set_ydata(data_np)

                # compute FFT and update line
                yf = fft(data_int)
                self.line_fft.set_ydata(
                    np.abs(yf[0:self.CHUNK]) / (128 * self.CHUNK))

                # update figure canvas
                self.fig.canvas.draw()
                self.fig.canvas.flush_events()
                frame_count += 1
        except:
            self.fr = frame_count / (time.time() - start_time)
            ser.close()
            print('average frame rate = {:.0f} FPS'.format(self.fr))
            self.exit_app()

    def exit_app(self):
        print('stream closed')
        self.p.close(self.stream)

    def onClick(self, event):
        self.pause = True
    
    def sample(self, size):
        i = 0
        data = []
        while i < size:
            data.append(int(ser.readline()))
            i += 1
        return data

In [146]:
ser = serial.Serial('COM11', 115200)

In [191]:
def sample():
    i = 0
    data = []
    while i < 256:
        data.append(try_float(ser.readline().decode("utf-8").strip('\n').strip('\r')))
        i += 1
    return data

def try_float(s):
    try:
        return float(s)
    except:
        return 0

In [167]:
ser.close()

In [207]:
ser = serial.Serial('COM11', 115200)
fig, (ax1, ax2) = plt.subplots(2, figsize=(15, 7))
x = np.arange(0, 256)
x_fft = np.linspace(0, 1000, 256)
line, = ax1.plot(x, np.random.rand(256), '-', lw=2)
line_fft, = ax2.plot(x_fft, np.random.rand(256), '-', lw=2)
ax1.set_ylim(0, 2)
ax1.set_xlim(0, 255)
ax2.set_ylim(0, 0.5)
ax2.set_xlim(10, 256)
try:
    while True:
        data = np.array(sample())
        line.set_ydata(data)
        y_fft = fft(data)
        line_fft.set_ydata(np.abs(y_fft)[0:256] * 2/256)
        fig.canvas.draw()
        fig.canvas.flush_events()
except Exception as e:
    print(e)
    ser.close()

invalid command name "."


In [193]:
ser.close()
ser = serial.Serial('COM11', 115200)
data = sample()
print(data)
ser.close()

[0.78, 0.71, 0, 0.53, 0.59, 0.69, 0.77, 0.8, 0.75, 0.65, 0.56, 0.53, 0.57, 0.66, 0.75, 0.8, 0.77, 0.68, 0.59, 0.53, 0.55, 0.63, 0.73, 0.79, 0.78, 0.72, 0.62, 0.54, 0.54, 0.6, 0.7, 0.78, 0.79, 0.74, 0.64, 0.56, 0.53, 0.58, 0.67, 0.76, 0.8, 0.76, 0.67, 0.57, 0.53, 0.56, 0.65, 0.74, 0.8, 0.78, 0.7, 0.6, 0.53, 0.54, 0.62, 0.72, 0.79, 0.79, 0.72, 0.62, 0.54, 0.53, 0.6, 0.7, 0.78, 0.79, 0.74, 0.64, 0.56, 0.53, 0.58, 0.67, 0.76, 0.8, 0.76, 0.67, 0.58, 0.53, 0.56, 0.65, 0.74, 0.8, 0.78, 0.7, 0.6, 0.53, 0.54, 0.62, 0.72, 0.79, 0.79, 0.72, 0.62, 0.54, 0.53, 0.6, 0.7, 0.78, 0.79, 0.74, 0.65, 0.56, 0.53, 0.58, 0.67, 0.76, 0.8, 0.76, 0.67, 0.58, 0.53, 0.56, 0.64, 0.74, 0.79, 0.78, 0.7, 0.6, 0.54, 0.54, 0.62, 0.72, 0.79, 0.79, 0.73, 0.63, 0.55, 0.53, 0.59, 0.69, 0.77, 0.8, 0.75, 0.65, 0.56, 0.53, 0.57, 0.66, 0.76, 0.8, 0.76, 0.68, 0.58, 0.53, 0.55, 0.63, 0.73, 0.79, 0.78, 0.71, 0.61, 0.54, 0.54, 0.61, 0.71, 0.78, 0.79, 0.73, 0.64, 0.55, 0.53, 0.58, 0.68, 0.76, 0.8, 0.76, 0.67, 0.57, 0.53, 0.56, 0.65

In [212]:
y_fft = fft(data)
sp_data = np.abs(y_fft)[0:128//2] * 2/256
plt.plot(sp_data)

In [213]:
np.linspace(0, 500, 128)

array([   0.        ,    3.93700787,    7.87401575,   11.81102362,
         15.7480315 ,   19.68503937,   23.62204724,   27.55905512,
         31.49606299,   35.43307087,   39.37007874,   43.30708661,
         47.24409449,   51.18110236,   55.11811024,   59.05511811,
         62.99212598,   66.92913386,   70.86614173,   74.80314961,
         78.74015748,   82.67716535,   86.61417323,   90.5511811 ,
         94.48818898,   98.42519685,  102.36220472,  106.2992126 ,
        110.23622047,  114.17322835,  118.11023622,  122.04724409,
        125.98425197,  129.92125984,  133.85826772,  137.79527559,
        141.73228346,  145.66929134,  149.60629921,  153.54330709,
        157.48031496,  161.41732283,  165.35433071,  169.29133858,
        173.22834646,  177.16535433,  181.1023622 ,  185.03937008,
        188.97637795,  192.91338583,  196.8503937 ,  200.78740157,
        204.72440945,  208.66141732,  212.5984252 ,  216.53543307,
        220.47244094,  224.40944882,  228.34645669,  232.28346